In [1]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Results Data Frame Analysis
df_Results = pd.DataFrame(columns=['Classification', 'ConfusionMatrix','Result'])

In [3]:
# Importing the dataset
dataset = pd.read_csv('./healthcare/train_data.csv')
dataset = dataset.dropna(how='any',axis=0)
dataset.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558.0,41-50


In [6]:
X = dataset.drop('case_id', axis=1)
X = X.drop('patientid', axis=1)
X = X.drop('Visitors with Patient', axis=1)
X = X.drop('Stay', axis=1)
X.head()

,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,City_Code_Patient,Type of Admission,Severity of Illness,Age,Admission_Deposit
0,8,c,3,Z,3,radiotherapy,R,F,2.0,7.0,Emergency,Extreme,51-60,4911.0
1,2,c,5,Z,2,radiotherapy,S,F,2.0,7.0,Trauma,Extreme,51-60,5954.0
2,10,e,1,X,2,anesthesia,S,E,2.0,7.0,Trauma,Extreme,51-60,4745.0
3,26,b,2,Y,2,radiotherapy,R,D,2.0,7.0,Trauma,Extreme,51-60,7272.0
4,26,b,2,Y,2,radiotherapy,S,D,2.0,7.0,Trauma,Extreme,51-60,5558.0


In [7]:
y = pd.DataFrame({'Stay': dataset['Stay']})
y.head()

,Stay
0,0-10
1,41-50
2,31-40
3,41-50
4,41-50


In [8]:
print(X.shape)
print(y.shape)

(313793, 14)
(313793, 1)


In [9]:
X = X.to_numpy()
y = y.to_numpy()

In [10]:
from sklearn.preprocessing import LabelEncoder

lblEncoder_Hospital_type_code = LabelEncoder()
X[:, 1] = lblEncoder_Hospital_type_code.fit_transform(X[:, 1])

lblEncoder_Hospital_region_code = LabelEncoder()
X[:, 3] = lblEncoder_Hospital_region_code.fit_transform(X[:, 3])

lblEncoder_Department = LabelEncoder()
X[:, 5] = lblEncoder_Department.fit_transform(X[:, 5])

lblEncoder_Ward_Type = LabelEncoder()
X[:, 6] = lblEncoder_Ward_Type.fit_transform(X[:, 6])

lblEncoder_Ward_Facility_Code = LabelEncoder()
X[:, 7] = lblEncoder_Ward_Facility_Code.fit_transform(X[:, 7])

lblEncoder_TypeofAdmission = LabelEncoder()
X[:, 10] = lblEncoder_TypeofAdmission.fit_transform(X[:, 10])

lblEncoder_SeverityofIllness = LabelEncoder()
X[:, 11] = lblEncoder_SeverityofIllness.fit_transform(X[:, 11])

lblEncoder_Age = LabelEncoder()
X[:, 12] = lblEncoder_Age.fit_transform(X[:, 12])

In [11]:
lblEncoder_Stay = LabelEncoder()
X[:, 0] = lblEncoder_Stay.fit_transform(X[:, 0])

In [12]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [13]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
###############################
# Decision Tree
###############################

# Training the Decision Tree Classification model on the Training set
from sklearn.tree import DecisionTreeClassifier
criterion = 'entropy'
dectree_classifier = DecisionTreeClassifier(criterion = criterion)
dectree_classifier.fit(X_train, y_train)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
dectree_y_pred = dectree_classifier.predict(X_test)
dectree_cm = confusion_matrix(y_test, dectree_y_pred)
dectree_acc = accuracy_score(y_test, dectree_y_pred)

df_Results.loc[len(df_Results)] = ['Decision Tree (criterion = ' + criterion + ')', dectree_cm, dectree_acc]

In [16]:
df_Results

,Classification,ConfusionMatrix,Result
0,Decision Tree (criterion = entropy),"[[837, 1572, 1281, 923, 210, 547, 47, 172, 85,...",0.234968


In [17]:
###############################
# K-Nearest Neighbors (K-NN)
###############################

# Training the K-NN model on the Training set
from sklearn.neighbors import KNeighborsClassifier
n_neighbors = 5
metric = 'minkowski'
p = 2
knn_classifier = KNeighborsClassifier(n_neighbors = n_neighbors, metric = metric, p = p)
knn_classifier.fit(X_train, y_train)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
knn_y_pred = knn_classifier.predict(X_test)
knn_cm = confusion_matrix(y_test, knn_y_pred)
knn_acc = accuracy_score(y_test, knn_y_pred)

df_Results.loc[len(df_Results)] = ['K-Nearest Neighbors (K-NN) (n_neighbors = ' + str(n_neighbors) + ', metric = ' + metric + ', p = ' + str(p) + ')', knn_cm, knn_acc]

d:\_production\dev.work\av-healthcare-analytics-ii\venv37\lib\site-packages\ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


In [18]:
df_Results

,Classification,ConfusionMatrix,Result
0,Decision Tree (criterion = entropy),"[[837, 1572, 1281, 923, 210, 547, 47, 172, 85,...",0.234968
1,"K-Nearest Neighbors (K-NN) (n_neighbors = 5, m...","[[966, 2296, 1475, 639, 46, 311, 4, 38, 8, 7, ...",0.280717


In [ ]:
###############################
# Kernel SVM
###############################

# Training the Kernel SVM model on the Training set
from sklearn.svm import SVC
kernel = 'rbf'
svm_classifier = SVC(kernel = kernel)
svm_classifier.fit(X_train, y_train)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
svm_y_pred = svm_classifier.predict(X_test)
svm_cm = confusion_matrix(y_test, svm_y_pred)
svm_acc = accuracy_score(y_test, svm_y_pred)

df_Results.loc[len(df_Results)] = ['Kernel SVM (kernel = ' + kernel + ')', svm_cm, svm_acc]

d:\_production\dev.work\av-healthcare-analytics-ii\venv37\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [ ]:
df_Results

## CONTINUE FROM HERE

In [ ]:
###############################
# Logistic Regression
###############################

## Training the Logistic Regression model on the Training set
from sklearn.linear_model import LogisticRegression
logreg_classifier = LogisticRegression()
logreg_classifier.fit(X_train, y_train)

## Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
logreg_y_pred = logreg_classifier.predict(X_test)
logreg_cm = confusion_matrix(y_test, logreg_y_pred)
logreg_acc = accuracy_score(y_test, logreg_y_pred)

df_Results.loc[len(df_Results)] = ['Logistic Regression', logreg_cm, logreg_acc]

In [ ]:
df_Results

In [ ]:
###############################
# Naive Bayes
###############################

## Training the Naive Bayes model on the Training set
from sklearn.naive_bayes import GaussianNB
naibay_classifier = GaussianNB()
naibay_classifier.fit(X_train, y_train)

## Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
naibay_y_pred = naibay_classifier.predict(X_test)
naibay_cm = confusion_matrix(y_test, naibay_y_pred)
naibay_acc = accuracy_score(y_test, naibay_y_pred)

df_Results.loc[len(df_Results)] = ['Naive Bayes', naibay_cm, naibay_acc]

In [ ]:
df_Results

In [ ]:
###############################
# Random Forest
###############################

## Training the Random Forest Classification model on the Training set
from sklearn.ensemble import RandomForestClassifier
n_estimators = 10
criterion = 'entropy'
randfor_classifier = RandomForestClassifier(n_estimators = n_estimators, criterion = criterion)
randfor_classifier.fit(X_train, y_train)

## Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
randfor_y_pred = randfor_classifier.predict(X_test)
randfor_cm = confusion_matrix(y_test, randfor_y_pred)
randfor_acc = accuracy_score(y_test, randfor_y_pred)


df_Results.loc[len(df_Results)] = ['Random Forest (n_estimators = ' + str(n_estimators) + ', criterion = ' + criterion + ')', randfor_cm, randfor_acc]

In [ ]:
df_Results

In [ ]:
###############################
# Support Vector Machine (SVM)
###############################

## Training the SVM model on the Training set
from sklearn.svm import SVC
kernel = 'linear'
suvema_classifier = SVC(kernel = kernel)
suvema_classifier.fit(X_train, y_train)

## Making the Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
suvema_y_pred = suvema_classifier.predict(X_test)
suvema_cm = confusion_matrix(y_test, suvema_y_pred)
suvema_acc = accuracy_score(y_test, suvema_y_pred)

df_Results.loc[len(df_Results)] = ['Support Vector Machine (SVM) (kernel = ' + kernel + ')', suvema_cm, suvema_acc]

In [ ]:
# Display Final Results
print(df_Results.sort_values(by=['Result'], ascending=False))